In [1]:
!pip install flask

In [2]:
# =================================================================================================
# Nombre: ArmaTuLogin+Romero 
# Autor: Jeshua Romero Guadarrama
# =================================================================================================
# Aplicación Flask local con:
# - Diccionarios para almacenar datos de usuarios
# - Base de datos local en archivo JSON (usuarios_db.json)
# - Rutas: inicio, registro, login, mostrar_usuarios
# - Renderizado de HTML con render_template_string (sin herencia de plantillas)
# - Uso de Bootstrap
# =================================================================================================

import os
import json
import threading
from flask import Flask, render_template_string, request

app = Flask(__name__)

# Nombre del archivo de la base de datos
NOMBRE_ARCHIVO_BD = "USUARIOS_BD.json"

# Diccionario para almacenar usuarios en memoria
usuarios = {}

# -------------------------------------------------------------------------------------------------------------------
# Funciones de carga y guardado de la base de datos
# -------------------------------------------------------------------------------------------------------------------
def cargar_db():
    """Carga los datos de usuarios desde un archivo JSON. Si no existe, crea uno vacío."""
    global usuarios
    if not os.path.exists(NOMBRE_ARCHIVO_BD):
        with open(NOMBRE_ARCHIVO_BD, 'w') as f:
            json.dump({}, f)
    else:
        with open(NOMBRE_ARCHIVO_BD, 'r') as f:
            usuarios = json.load(f)

def guardar_db():
    """Guarda el diccionario de usuarios en un archivo JSON."""
    with open(NOMBRE_ARCHIVO_BD, 'w') as f:
        json.dump(usuarios, f, indent=4)

# -------------------------------------------------------------------------------------------------------------------
# Funciones para registrar y validar usuarios
# -------------------------------------------------------------------------------------------------------------------
def registrar_usuario(usuario, contrasena):
    """Registra un nuevo usuario si no existe. Devuelve True si se registró, False si ya existía."""
    if usuario in usuarios:
        return False
    else:
        usuarios[usuario] = contrasena
        guardar_db()
        return True

def login_usuario(usuario, contrasena):
    """Verifica si el usuario existe y la contraseña es correcta."""
    if usuario in usuarios:
        return usuarios[usuario] == contrasena
    return False

# -------------------------------------------------------------------------------------------------------------------
# Función para obtener todos los usuarios
# -------------------------------------------------------------------------------------------------------------------
def obtener_todos_los_usuarios():
    return usuarios

# -------------------------------------------------------------------------------------------------------------------
# HTML “base” embebido en cada ruta (sin herencia de plantillas)
# -------------------------------------------------------------------------------------------------------------------
def envolver_html(content, title = "Finanzas Sostenibles"):
    """
    Envuelve una cadena 'content' dentro de una estructura HTML con Bootstrap.
    """
    return f"""
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="UTF-8">
  <title>{title}</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.6.2/dist/css/bootstrap.min.css">
</head>
<body>

<nav class="navbar navbar-expand-lg navbar-dark bg-dark">
  <a class="navbar-brand" href="/">Finanzas Sostenibles</a>
  <div class="collapse navbar-collapse">
    <ul class="navbar-nav ml-auto">
      <li class="nav-item">
        <a class="nav-link" href="/registro">Registro</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" href="/login">Login</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" href="/mostrar_usuarios">Usuarios</a>
      </li>
    </ul>
  </div>
</nav>

<div class="container mt-4">
  {content}
</div>

<footer class="footer bg-light text-center text-lg-start mt-4">
  <div class="text-center p-3" style="background-color: #f8f9fa;">
    © 2025 - Página de ejemplo creada por: <strong>Jeshua Romero Guadarrama</strong>
  </div>
</footer>

<script src="https://cdn.jsdelivr.net/npm/jquery@3.5.1/dist/jquery.slim.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/bootstrap@4.6.2/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>
"""

# -------------------------------------------------------------------------------------------------------------------
# Ruta: Inicio
# -------------------------------------------------------------------------------------------------------------------
@app.route('/')
def index():
    """
    Muestra información sobre la sostenibilidad y finanzas en México.
    """
    content = """
    <div class="jumbotron">
      <h1 class="display-4">Sostenibilidad y Finanzas en México</h1>
      <p class="lead">
        La convergencia entre tecnología financiera (Fintech), neobancos 
        y desarrollo de software sustentable es fundamental para impulsar 
        la inclusión financiera, reducir la brecha social y, al mismo tiempo, 
        cuidar el medio ambiente.
      </p>
      <hr class="my-4">
      <p>
        En México, las Fintech y los neobancos ofrecen soluciones bancarias 
        digitales que reducen costos operativos y fomentan la competitividad. 
        Con inversiones en energías renovables, préstamos verdes y plataformas 
        de pago más eficientes, se potencia la economía local, se promueve la 
        justicia social y se minimiza el impacto ambiental.
      </p>
      <p>
        Este proyecto es un ejemplo de cómo combinar la innovación tecnológica 
        (registro de usuarios, base de datos local) con la promoción de la 
        sostenibilidad. ¡Bienvenido!
      </p>
    </div>
    """
    return render_template_string(envolver_html(content, "Inicio"))

# -------------------------------------------------------------------------------------------------------------------
# Ruta: Registro
# -------------------------------------------------------------------------------------------------------------------
@app.route('/registro', methods = ['GET', 'POST'])
def registro():
    if request.method == 'POST':
        usuario = request.form.get('usuario')
        contrasena = request.form.get('contrasena')

        if usuario and contrasena:
            if registrar_usuario(usuario, contrasena):
                msg = """
                <h3>Usuario registrado con éxito.</h3>
                <p><a href="/">Volver a la página de inicio</a></p>
                """
            else:
                msg = """
                <h3>Error: El usuario ya existe.</h3>
                <p><a href="/registro">Intentar de nuevo</a></p>
                """
        else:
            msg = """
            <h3>Error: Datos incompletos.</h3>
            <p><a href="/registro">Intentar de nuevo</a></p>
            """
        return render_template_string(envolver_html(msg, "Registro"))

    # Si es método GET, mostramos el formulario
    form_html = """
    <div class="row">
      <div class="col-md-6 offset-md-3">
        <h2>Registro de Usuario</h2>
        <form method="POST" action="/registro">
          <div class="form-group">
            <label for="usuario">Nombre de Usuario</label>
            <input type="text" class="form-control" id="usuario" name="usuario" required>
          </div>
          <div class="form-group">
            <label for="contrasena">Contraseña</label>
            <input type="password" class="form-control" id="contrasena" name="contrasena" required>
          </div>
          <button type="submit" class="btn btn-primary">Registrar</button>
        </form>
      </div>
    </div>
    """
    return render_template_string(envolver_html(form_html, "Registro"))

# -------------------------------------------------------------------------------------------------------------------
# Ruta: Login
# -------------------------------------------------------------------------------------------------------------------
@app.route('/login', methods = ['GET', 'POST'])
def login():
    if request.method == 'POST':
        usuario = request.form.get('usuario')
        contrasena = request.form.get('contrasena')

        if usuario and contrasena:
            if login_usuario(usuario, contrasena):
                msg = f"""
                <h3>¡Bienvenido, {usuario}!</h3>
                <p>Login exitoso.</p>
                <p><a href="/">Ir a inicio</a></p>
                """
            else:
                msg = """
                <h3>Error: Usuario o contraseña incorrectos.</h3>
                <p><a href="/login">Intentar de nuevo</a></p>
                """
        else:
            msg = """
            <h3>Error: Datos incompletos.</h3>
            <p><a href="/login">Intentar de nuevo</a></p>
            """
        return render_template_string(envolver_html(msg, "Login"))

    # Si es método GET, mostramos el formulario de login
    form_html = """
    <div class="row">
      <div class="col-md-6 offset-md-3">
        <h2>Inicio de Usuario</h2>
        <form method="POST" action="/login">
          <div class="form-group">
            <label for="usuario">Nombre de Usuario</label>
            <input type="text" class="form-control" id="usuario" name="usuario" required>
          </div>
          <div class="form-group">
            <label for="contrasena">Contraseña</label>
            <input type="password" class="form-control" id="contrasena" name="contrasena" required>
          </div>
          <button type="submit" class="btn btn-success">Iniciar Sesión</button>
        </form>
      </div>
    </div>
    """
    return render_template_string(envolver_html(form_html, "Login"))

# -------------------------------------------------------------------------------------------------------------------
# Ruta: Mostrar Usuarios Registrados
# -------------------------------------------------------------------------------------------------------------------
@app.route('/mostrar_usuarios')
def mostrar_usuarios():
    todos = obtener_todos_los_usuarios()
    if todos:
        table_rows = ""
        for usuario, contrasena in todos.items():
            table_rows += f"""
            <tr>
              <td>{usuario}</td>
              <td>{contrasena}</td>
            </tr>
            """
        content = f"""
        <h2>Lista de Usuarios Registrados</h2>
        <table class="table table-bordered table-striped">
          <thead>
            <tr>
              <th>Usuario</th>
              <th>Contraseña</th>
            </tr>
          </thead>
          <tbody>
            {table_rows}
          </tbody>
        </table>
        """
    else:
        content = """
        <h2>Lista de Usuarios Registrados</h2>
        <p>No hay usuarios registrados.</p>
        """
    return render_template_string(envolver_html(content, "Usuarios Registrados"))

# -------------------------------------------------------------------------------------------------------------------
# Función para correr el servidor en un hilo aparte
# -------------------------------------------------------------------------------------------------------------------
def run_server():
    cargar_db()  # Carga la base de datos al iniciar
    print("Servidor Flask corriendo en http://127.0.0.1:5000")
    app.run(debug = False, use_reloader = False)  # Se desactiva reloader para evitar doble ejecución en Jupyter

# -------------------------------------------------------------------------------------------------------------------
# Iniciar el servidor en segundo plano
# -------------------------------------------------------------------------------------------------------------------
# Esto permitirá que la celda termine, pero el server siga ejecutándose mientras el kernel esté activo.
server_thread = threading.Thread(target = run_server)
server_thread.start()

print('¡Listo! El servidor Flask se está ejecutando en segundo plano.')
print('Visita http://127.0.0.1:5000 en tu navegador (si estás en local).')

¡Listo! El servidor Flask se está ejecutando en segundo plano.
Visita http://127.0.0.1:5000 en tu navegador (si estás en local).
Servidor Flask corriendo en http://127.0.0.1:5000
